In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pysindy import SINDy
from pysindy.optimizers import STLSQ, ConstrainedSR3
from pysindy.feature_library import PolynomialLibrary
from sklearn.metrics import mean_squared_error
from scipy.ndimage import gaussian_filter1d
import warnings

warnings.filterwarnings("ignore")

# Functions from your code
def apply_gaussian_filter(data, sigma):
    return gaussian_filter1d(data, sigma=sigma)

def integrate_data(data, dt=1):
    integrated_data = np.zeros(len(data))
    for i in range(1, len(data)):
        integrated_data[i] = integrated_data[i-1] + data[i] * dt 
    return integrated_data

def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))



In [8]:
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy.integrate import cumtrapz
from sklearn.metrics import mean_squared_error
from pysindy import SINDy, PolynomialLibrary, STLSQ, ConstrainedSR3
import pickle
import os

# Existing functions (unchanged)
def apply_gaussian_filter(data, sigma):
    return gaussian_filter1d(data, sigma=sigma)

def integrate_data(omega_data, dt=1, initial_theta=0):
    theta_data = cumtrapz(omega_data, dx=dt, initial=initial_theta)
    return theta_data

def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Optimized function: Train original model once per chunk
def analyze_constrained_sindy(filtered_data, sigma=60, degree=2, divergence_threshold=0.5, transform_name="linear_time"):
    feature_names = ['1', 'theta', 'omega', 'time', 'theta^2', 'theta omega', 
                     'theta time', 'omega^2', 'omega time', 'time^2']
    n_features = len(feature_names)

    s = 1
    delta_t = 1 / s
    simulation_steps = 3600
    poly_library = PolynomialLibrary(degree=degree)
    original_optimizer = STLSQ(threshold=1e-10)

    # Store results
    results = {
        'constraint_index': [],
        'constrained_term': [],
        'mean_original_rmse': [],
        'mean_constrained_rmse': [],
        'unstable_original': [],
        'unstable_constrained': [],
        'total_chunks': []
    }
    all_rmse_data = {}
    all_omega_data = {}

    # Precompute original model results for all chunks
    original_results = []
    total_chunks = 0
    print("Precomputing unconstrained model for all chunks...")
    for i, chunk in enumerate(filtered_data):
        total_chunks += 1
        omega_filtered = apply_gaussian_filter(chunk['omega'].values, sigma=sigma)
        chunk['omega_filtered'] = omega_filtered
        theta_chunk = integrate_data(chunk['omega_filtered'].values)
        stacked_data_chunk = np.column_stack((theta_chunk, chunk['omega_filtered'].values))
        t_train_chunk = np.arange(0, len(chunk), 1).reshape(-1, 1)
        x_train_augmented_chunk = np.hstack([stacked_data_chunk, t_train_chunk])

        true_omega = chunk['omega'].values
        time_points = np.arange(0, simulation_steps, 1) * delta_t

        # Fit original model once per chunk
        original_model = SINDy(feature_names=feature_names, 
                               feature_library=poly_library, 
                               optimizer=original_optimizer)
        original_model.fit(x_train_augmented_chunk, t=1)
        original_coefficients = original_model.coefficients()

        th = np.zeros(simulation_steps)
        om_original = np.zeros(simulation_steps)
        t = np.zeros(simulation_steps)
        th[0], om_original[0], t[0] = theta_chunk[0], true_omega[0], 0 
        limited_simulation = False

        for step in range(simulation_steps - 1):
            state = np.array([th[step], om_original[step], t[step]])
            features = poly_library.fit_transform(state.reshape(1, -1))[0]
            derivatives = original_coefficients @ features
            th[step + 1] = th[step] + delta_t * derivatives[0]
            om_original[step + 1] = om_original[step] + delta_t * derivatives[1]
            t[step + 1] = t[step] + delta_t
            if np.abs(om_original[step + 1]) > divergence_threshold:
                limited_simulation = True
                simulation_end = step + 2
                break

        om_original_used = om_original[:simulation_end] if limited_simulation else om_original
        true_omega_used = true_omega[:len(om_original_used)]
        rmse_original = calculate_rmse(true_omega_used, om_original_used)
        print(f"Chunk {i} RMSE (original): {rmse_original:.6f}")

        original_results.append({
            'omega_filtered': omega_filtered,
            'true_omega': true_omega,
            'theta_chunk': theta_chunk,
            'x_train_augmented_chunk': x_train_augmented_chunk,
            'om_original': om_original.copy(),
            'time_points': time_points[:len(om_original)] if limited_simulation else time_points.copy(),
            'rmse_original': rmse_original,
            'unstable': limited_simulation
        })

    # Process constraints using precomputed original results
    constraint_indices = range(n_features, 2 * n_features)
    constrained_terms = feature_names

    for constraint_idx, constrained_term in zip(constraint_indices, constrained_terms):
        original_rmse = []
        constrained_rmse = []
        unstable_original = 0
        unstable_constrained = 0

        all_omega_data[constrained_term] = {}
        all_rmse_data[constrained_term] = {'original': [], 'constrained': []}

        constraint_lhs = np.zeros((1, 3 * n_features))
        constraint_lhs[0, constraint_idx] = 1
        constraint_rhs = np.array([0])
        constrained_optimizer = ConstrainedSR3(
            threshold=1e-10,
            max_iter=100,
            constraint_lhs=constraint_lhs,
            constraint_rhs=constraint_rhs,
            constraint_order="feature"
        )

        print(f"\nProcessing constraint on '{constrained_term}' (index {constraint_idx})")

        for i, chunk_data in enumerate(original_results):
            # Reuse precomputed original results
            omega_filtered = chunk_data['omega_filtered']
            true_omega = chunk_data['true_omega']
            theta_chunk = chunk_data['theta_chunk']
            x_train_augmented_chunk = chunk_data['x_train_augmented_chunk']
            om_original = chunk_data['om_original']
            time_points = chunk_data['time_points']
            rmse_original = chunk_data['rmse_original']
            unstable = chunk_data['unstable']

            original_rmse.append(rmse_original)
            all_rmse_data[constrained_term]['original'].append(rmse_original)
            if unstable:
                unstable_original += 1

            # Constrained model
            constrained_model = SINDy(feature_names=feature_names, 
                                      feature_library=poly_library, 
                                      optimizer=constrained_optimizer)
            constrained_model.fit(x_train_augmented_chunk, t=1)
            constrained_coefficients = constrained_model.coefficients()

            th = np.zeros(simulation_steps)
            om_constrained = np.zeros(simulation_steps)
            t = np.zeros(simulation_steps)
            th[0], om_constrained[0], t[0] = theta_chunk[0], omega_filtered[0], 0
            limited_simulation = False

            for step in range(simulation_steps - 1):
                state = np.array([th[step], om_constrained[step], t[step]])
                features = poly_library.fit_transform(state.reshape(1, -1))[0]
                derivatives = constrained_coefficients @ features
                th[step + 1] = th[step] + delta_t * derivatives[0]
                om_constrained[step + 1] = om_constrained[step] + delta_t * derivatives[1]
                t[step + 1] = t[step] + delta_t
                if np.abs(om_constrained[step + 1]) > divergence_threshold:
                    limited_simulation = True
                    simulation_end = step + 2
                    unstable_constrained += 1
                    break

            om_constrained_used = om_constrained[:simulation_end] if limited_simulation else om_constrained
            true_omega_used = true_omega[:len(om_constrained_used)]
            rmse_constrained = calculate_rmse(true_omega_used, om_constrained_used)
            constrained_rmse.append(rmse_constrained)
            all_rmse_data[constrained_term]['constrained'].append(rmse_constrained)
            print(f"Chunk {i} RMSE (constrained): {rmse_constrained:.6f}")

            all_omega_data[constrained_term][i] = {
                'true': true_omega.copy(),
                'filtered': omega_filtered.copy(),
                'original': om_original.copy(),
                'constrained': om_constrained.copy(),
                'time': time_points.copy()
            }

        # Aggregate results
        mean_original_rmse = np.mean(original_rmse)
        mean_constrained_rmse = np.mean(constrained_rmse)

        results['constraint_index'].append(constraint_idx)
        results['constrained_term'].append(constrained_term)
        results['mean_original_rmse'].append(mean_original_rmse)
        results['mean_constrained_rmse'].append(mean_constrained_rmse)
        results['unstable_original'].append(unstable_original)
        results['unstable_constrained'].append(unstable_constrained)
        results['total_chunks'].append(total_chunks)

        print(f"Completed constraint on {constrained_term}: Mean RMSE (original) = {mean_original_rmse:.6f}, "
              f"Mean RMSE (constrained) = {mean_constrained_rmse:.6f}, "
              f"Unstable (original) = {unstable_original}/{total_chunks}, "
              f"Unstable (constrained) = {unstable_constrained}/{total_chunks}")

    # Save all results
    output_dir = "C:/1. Power grid frequency data/results/"
    os.makedirs(output_dir, exist_ok=True)
    with open(f"{output_dir}constrained_sindy_results.pkl", 'wb') as f:
        pickle.dump(results, f)
    with open(f"{output_dir}all_omega_data.pkl", 'wb') as f:
        pickle.dump(all_omega_data, f)
    with open(f"{output_dir}all_rmse_data.pkl", 'wb') as f:
        pickle.dump(all_rmse_data, f)
    print("Results, omega data, and RMSE data saved.")

    return results, all_omega_data, all_rmse_data



In [11]:
import numpy as np
print(np.__version__)


1.26.4


In [10]:
# Load and preprocess data
data = pd.read_pickle("C:/1. Power grid frequency data/df_South_Korea_cleansed_2024-08-15_2024-12-10.pkl")
data.index = pd.to_datetime(data.index)
data_filtered = data[(data['QI'] == 0) & (data['freq'].notna())].drop(columns=['QI']).dropna()
hourly_groups = data_filtered.groupby(data_filtered.index.floor('H'))
valid_hours = hourly_groups.filter(lambda x: len(x) == 3600)
valid_hours['omega'] = 2 * np.pi * (valid_hours['freq'] - 60)

# Split data into hourly chunks
region_data_list = [group for _, group in valid_hours.groupby(valid_hours.index.floor('H'))]
print(f"Total number of valid hourly chunks: {len(region_data_list)}")

# Run the analysis
results, all_omega_data, all_rmse_data = analyze_constrained_sindy(region_data_list, sigma=60, degree=2, divergence_threshold=0.5)

Total number of valid hourly chunks: 2451
Precomputing unconstrained model for all chunks...
Chunk 0 RMSE (original): 0.113620
Chunk 1 RMSE (original): 0.181735
Chunk 2 RMSE (original): 0.098623
Chunk 3 RMSE (original): 0.121168
Chunk 4 RMSE (original): 0.091556
Chunk 5 RMSE (original): 0.087337
Chunk 6 RMSE (original): 0.087353
Chunk 7 RMSE (original): 0.079767
Chunk 8 RMSE (original): 0.292695
Chunk 9 RMSE (original): 0.143719
Chunk 10 RMSE (original): 0.232224
Chunk 11 RMSE (original): 0.079796
Chunk 12 RMSE (original): 0.236714
Chunk 13 RMSE (original): 0.095310
Chunk 14 RMSE (original): 0.109452
Chunk 15 RMSE (original): 0.143214
Chunk 16 RMSE (original): 0.072736
Chunk 17 RMSE (original): 0.144127
Chunk 18 RMSE (original): 0.208363
Chunk 19 RMSE (original): 0.209158
Chunk 20 RMSE (original): 0.077017
Chunk 21 RMSE (original): 0.108578
Chunk 22 RMSE (original): 0.103286
Chunk 23 RMSE (original): 0.061413
Chunk 24 RMSE (original): 0.090188
Chunk 25 RMSE (original): 0.084156
Chunk 2

C:\Users\wp7528\AppData\Roaming\Python\Python310\site-packages\pysindy\optimizers\constrained_sr3.py:463: ConvergenceWarning: SR3._reduce did not converge after 30 iterations.
  warnings.warn(


Chunk 0 RMSE (constrained): 0.146593


C:\Users\wp7528\AppData\Roaming\Python\Python310\site-packages\pysindy\optimizers\constrained_sr3.py:463: ConvergenceWarning: SR3._reduce did not converge after 30 iterations.
  warnings.warn(


Chunk 1 RMSE (constrained): 0.132548


...

C:\Users\wp7528\AppData\Roaming\Python\Python310\site-packages\pysindy\optimizers\constrained_sr3.py:463: ConvergenceWarning: SR3._reduce did not converge after 30 iterations.
  warnings.warn(


Chunk 358 RMSE (constrained): 0.109425


C:\Users\wp7528\AppData\Roaming\Python\Python310\site-packages\pysindy\optimizers\constrained_sr3.py:463: ConvergenceWarning: SR3._reduce did not converge after 30 iterations.
  warnings.warn(


Chunk 359 RMSE (constrained): 0.109515


KeyboardInterrupt: 

In [ ]:
# Optional: Save results
import pickle
output_dir = "C:/1. Power grid frequency data/results/"
os.makedirs(output_dir, exist_ok=True)
with open(f"{output_dir}constrained_sindy_results.pkl", 'wb') as f:
    pickle.dump(results, f)
with open(f"{output_dir}all_omega_data.pkl", 'wb') as f:
    pickle.dump(all_omega_data, f)
print("Results saved.")

In [ ]:
# CE Data
ce_path = "C:/1. Power grid frequency data/CE/2024.csv"
data_ce = pd.read_csv(ce_path, header=None, names=['Time', 'freq'])

# Convert Time column to datetime and set as index
data_ce['Time'] = pd.to_datetime(data_ce['Time'])
data_ce.set_index('Time', inplace=True)
data_ce['freq'] = pd.to_numeric(data_ce['freq'])

# Print rows with NA in CE before filtering
print("Rows with NA in CE data before filtering:")
print(data_ce[data_ce.isna().any(axis=1)])

# South Korea Data
data_sk = pd.read_pickle("C:/1. Power grid frequency data/df_South_Korea_cleansed_2024-08-15_2024-12-10.pkl")
data_sk.index = pd.to_datetime(data_sk.index)

# Print rows with NA in South Korea before filtering
print("\nRows with NA in South Korea data before filtering:")
print(data_sk[data_sk.isna().any(axis=1)])

# CE Filtering
data_filtered_ce = data_ce.dropna(subset=['freq'])
valid_hours_ce = data_filtered_ce.groupby(data_filtered_ce.index.floor('H')).filter(lambda x: len(x) == 3600)
valid_hours_ce['omega'] = 2 * np.pi * (valid_hours_ce['freq'] - 50)  # 50 Hz base for CE

# South Korea Filtering
data_filtered_sk = data_sk[(data_sk['QI'] == 0) & (data_sk['freq'].notna())].dropna()
valid_hours_sk = data_filtered_sk.groupby(data_filtered_sk.index.floor('H')).filter(lambda x: len(x) == 3600)
valid_hours_sk['omega'] = 2 * np.pi * (valid_hours_sk['freq'] - 60)  # 60 Hz base for South Korea

In [ ]:
# Apply Gaussian filter to the omega data
filtered_data_ce = apply_gaussian_filter(valid_hours_ce['omega'], 'CE')
filtered_data_sk = apply_gaussian_filter(valid_hours_sk['omega'], 'SK')

In [ ]:
# Feature names
feature_names = ['1', 'theta', 'omega', 'time', 'theta^2', 'theta omega', 
                 'theta time', 'omega^2', 'omega time', 'time^2']
n_features = len(feature_names)

# Simulation parameters
s = 1  # Sampling rate (1 Hz)
delta_t = 1 / s
simulation_steps = 3600
divergence_threshold = 0.5

# Single sigma value
sigma = 60

# Polynomial library
poly_library = PolynomialLibrary(degree=2)

# Optimizers for original model (unchanged)
original_optimizer = STLSQ(threshold=1e-10)

# Store results for each constraint
results = {
    'constraint_index': [],
    'constrained_term': [],
    'mean_original_rmse': [],
    'mean_constrained_rmse': [],
    'unstable_original': [],
    'unstable_constrained': [],
    'total_chunks': []
}

# Dictionary to store all omega values: {constraint_term: {chunk_idx: {'true': ..., 'filtered': ..., 'original': ..., 'constrained': ..., 'time': ...}}}
all_omega_data = {}

# Loop over constraint indices from n_features + 3 to n_features + 9
constraint_indices = range(n_features , n_features + 10)  # 13 to 19
constrained_terms = feature_names[0:10]  # Corresponding terms: 'time' to 'time^2'

for constraint_idx, constrained_term in zip(constraint_indices, constrained_terms):
    # Initialize lists for this constraint case
    original_rmse = []
    constrained_rmse = []
    original_coeffs_all = []
    constrained_coeffs_all = []
    unstable_original = 0
    unstable_constrained = 0
    total_chunks = 0

    # Initialize storage for this constraint term
    all_omega_data[constrained_term] = {}

    # Set up constrained optimizer for this specific term
    constraint_lhs = np.zeros((1, 3 * n_features))  # Single constraint
    constraint_lhs[0, constraint_idx] = 1  # Constrain one term in domega/dt
    constraint_rhs = np.array([0])
    constrained_optimizer = ConstrainedSR3(
        threshold=1e-10,
        constraint_lhs=constraint_lhs,
        constraint_rhs=constraint_rhs,
        constraint_order="feature"
    )

    # Suppress numerical warnings
    np.seterr(over='raise', divide='raise', invalid='raise')

    # Loop over all chunks in filtered_data_sk
    for i, chunk in enumerate(filtered_data_sk):
        total_chunks += 1

        # Apply Gaussian filter with sigma=60
        omega_filtered = apply_gaussian_filter(chunk['omega'].values, sigma=sigma)
        chunk['omega_filtered'] = omega_filtered

        # Prepare data for SINDy
        theta_chunk = integrate_data(chunk['omega_filtered'].values)
        stacked_data_chunk = np.column_stack((theta_chunk, chunk['omega_filtered'].values))
        t_train_chunk = np.arange(0, len(chunk), 1).reshape(-1, 1)
        x_train_augmented_chunk = np.hstack([stacked_data_chunk, t_train_chunk])

        # True omega for comparison
        true_omega = chunk['omega'].values

        # Time points
        time_points = np.arange(0, simulation_steps, 1) * delta_t

        # Fit original model (STLSQ) for 3 states
        original_model = SINDy(feature_names=feature_names, 
                               feature_library=poly_library, 
                               optimizer=original_optimizer)
        original_model.fit(x_train_augmented_chunk, t=1)
        original_coefficients = original_model.coefficients()

        # Manual simulation for original model
        th = np.zeros(simulation_steps)
        om_original = np.zeros(simulation_steps)
        t = np.zeros(simulation_steps)
        th[0], om_original[0], t[0] = theta_chunk[0], true_omega[0], 0 
        limited_simulation = False

        try:
            for step in range(simulation_steps - 1):
                state = np.array([th[step], om_original[step], t[step]])
                features = poly_library.fit_transform(state.reshape(1, -1))[0]
                derivatives = original_coefficients @ features

                th[step + 1] = th[step] + delta_t * derivatives[0]
                om_original[step + 1] = om_original[step] + delta_t * derivatives[1]
                t[step + 1] = t[step] + delta_t * derivatives[2]

                if np.abs(om_original[step + 1]) > divergence_threshold:
                    limited_simulation = True
                    simulation_end = step + 2
                    unstable_original += 1
                    break
        except (FloatingPointError, OverflowError, RuntimeWarning):
            limited_simulation = True
            simulation_end = 3600
            unstable_original += 1

        om_original_used = om_original[:simulation_end] if limited_simulation else om_original
        true_omega_used = true_omega[:len(om_original_used)]
        rmse_original = calculate_rmse(true_omega_used, om_original_used)
        original_rmse.append(rmse_original)
        original_coeffs_all.append(original_coefficients[1])

        # Fit constrained model
        constrained_model = SINDy(feature_names=feature_names, 
                                  feature_library=poly_library, 
                                  optimizer=constrained_optimizer)
        constrained_model.fit(x_train_augmented_chunk, t=1)
        constrained_coefficients = constrained_model.coefficients()

        # Manual simulation for constrained model
        th = np.zeros(simulation_steps)
        om_constrained = np.zeros(simulation_steps)
        t = np.zeros(simulation_steps)
        th[0], om_constrained[0], t[0] = theta_chunk[0], omega_filtered[0], 0
        limited_simulation = False

        try:
            for step in range(simulation_steps - 1):
                state = np.array([th[step], om_constrained[step], t[step]])
                features = poly_library.fit_transform(state.reshape(1, -1))[0]
                derivatives = constrained_coefficients @ features

                th[step + 1] = th[step] + delta_t * derivatives[0]
                om_constrained[step + 1] = om_constrained[step] + delta_t * derivatives[1]
                t[step + 1] = t[step] + delta_t * derivatives[2]

                if np.abs(om_constrained[step + 1]) > divergence_threshold:
                    limited_simulation = True
                    simulation_end = step + 2
                    unstable_constrained += 1
                    break
        except (FloatingPointError, OverflowError, RuntimeWarning):
            limited_simulation = True
            simulation_end = 3600
            unstable_constrained += 1

        om_constrained_used = om_constrained[:simulation_end] if limited_simulation else om_constrained
        true_omega_used = true_omega[:len(om_constrained_used)]
        rmse_constrained = calculate_rmse(true_omega_used, om_constrained_used)
        constrained_rmse.append(rmse_constrained)
        constrained_coeffs_all.append(constrained_coefficients[1])

        # Store all omega values for this chunk and constraint
        all_omega_data[constrained_term][i] = {
            'true': true_omega.copy(),
            'filtered': omega_filtered.copy(),
            'original': om_original.copy(),
            'constrained': om_constrained.copy(),
            'time': time_points[:len(om_original)] if limited_simulation else time_points.copy()
        }

    # Aggregate results for this constraint
    mean_original_rmse = np.mean(original_rmse)
    mean_constrained_rmse = np.mean(constrained_rmse)

    # Store results
    results['constraint_index'].append(constraint_idx)
    results['constrained_term'].append(constrained_term)
    results['mean_original_rmse'].append(mean_original_rmse)
    results['mean_constrained_rmse'].append(mean_constrained_rmse)
    results['unstable_original'].append(unstable_original)
    results['unstable_constrained'].append(unstable_constrained)
    results['total_chunks'].append(total_chunks)

In [ ]:
# Print results for all constraints
print("\nResults for each constrained term in domega/dt (sigma=60):")
for i in range(len(results['constraint_index'])):
    idx = results['constraint_index'][i]
    term = results['constrained_term'][i]
    orig_rmse = results['mean_original_rmse'][i]
    const_rmse = results['mean_constrained_rmse'][i]
    unst_orig = results['unstable_original'][i]
    unst_const = results['unstable_constrained'][i]
    total = results['total_chunks'][i]
    print(f"\nConstrained term: {term} (index {idx})")
    print(f"Mean Original RMSE: {orig_rmse}")
    print(f"Mean Constrained RMSE: {const_rmse}")
    print(f"Number of unstable original chunks: {unst_orig}")
    print(f"Share of unstable original chunks: {unst_orig / total:.2%}")
    print(f"Number of unstable constrained chunks: {unst_const}")
    print(f"Share of unstable constrained chunks: {unst_const / total:.2%}")